In [1]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np 
import matplotlib.pyplot as plt 


Regularization

In [3]:
[name for name in dir(keras.regularizers) if not name.startswith('_')]

['L1',
 'L1L2',
 'L2',
 'OrthogonalRegularizer',
 'Regularizer',
 'deserialize',
 'get',
 'l1',
 'l1_l2',
 'l2',
 'orthogonal_regularizer',
 'serialize']

In [4]:
rL1 = keras.regularizers.l1(l1=.01)
rL2 = keras.regularizers.l2(l2=.01)
rL1L2 = keras.regularizers.l1_l2(l1=.01, l2=.01)

Ex1

In [5]:
#Load date
from tensorflow.keras.datasets import fashion_mnist
(xTrainSet, yTrainSet), (xTest, yTest) = fashion_mnist.load_data()

#Split date
from sklearn.model_selection import train_test_split
xTrain, xValid, yTrain, yValid = train_test_split(xTrainSet, yTrainSet, random_state=1)

#Preprocessing
pixelMeans = xTrain.mean(axis=0, keepdims=True)
pixelStds = xTrain.std(axis=0, keepdims=True)
xTrainScaled = (xTrain - pixelMeans) / pixelStds
xValidScaled = (xValid - pixelMeans) / pixelStds
xTestScaled = (xTest - pixelMeans) / pixelStds

In [6]:
tf.keras.backend.clear_session()
np.random.seed(1)
tf.random.set_seed(1)

In [8]:
rL2 = keras.regularizers.l2(l2=.01)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu', kernel_initializer='he_normal',kernel_regularizer=rL2),
    keras.layers.Dense(100, activation='relu', kernel_initializer='he_normal',kernel_regularizer=rL2),
    keras.layers.Dense(10, activation='softmax', kernel_regularizer= rL2)
])

In [10]:
model.compile (loss = 'sparse_categorical_crossentropy', optimizer ='nadam', metrics = ['accuracy'])

In [11]:
train = model.fit(xTrainScaled, yTrain, epochs=2, validation_data=(xValidScaled, yValid))

Epoch 1/2


2024-02-15 18:53:47.372909: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1407/1407 [==============================] - 3s 2ms/step - loss: 1.6378 - accuracy: 0.8157 - val_loss: 0.8427 - val_accuracy: 0.7891
Epoch 2/2
1407/1407 [==============================] - 3s 2ms/step - loss: 0.7343 - accuracy: 0.8277 - val_loss: 0.7216 - val_accuracy: 0.8294


Dropout

In [12]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(300, activation='relu', kernel_initializer='he_normal'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(100, activation='relu', kernel_initializer='he_normal'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

In [13]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [15]:
train = model.fit(xTrainScaled, yTrain, epochs=2 ,validation_data=(xValidScaled, yValid))

Epoch 1/2
1407/1407 [==============================] - 3s 2ms/step - loss: 0.5877 - accuracy: 0.7932 - val_loss: 0.4297 - val_accuracy: 0.8392
Epoch 2/2
1407/1407 [==============================] - 3s 2ms/step - loss: 0.4388 - accuracy: 0.8422 - val_loss: 0.3647 - val_accuracy: 0.8662


In [16]:
model.evaluate(xTestScaled, yTest)

313/313 [==============================] - 0s 451us/step - loss: 0.3876 - accuracy: 0.8563


[0.38756951689720154, 0.8562999963760376]

In [17]:
yProba = model.predict(xTestScaled)
yProba[:3].round(2)

313/313 [==============================] - 0s 367us/step


array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.1 , 0.  , 0.88],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]],
      dtype=float32)

In [18]:
yPred = np.argmax(yProba, axis=1)
yPred[:3]

array([9, 2, 1])

In [19]:
yTest[:3]

array([9, 2, 1], dtype=uint8)

MC Dropout

In [22]:
yProba = np.stack([model(xTestScaled[:3], training=True) for _ in range(100)])
yProba.shape

(100, 3, 10)

In [24]:
yProbaMc = yProba.mean(axis=0)
np.round(yProbaMc, 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.06, 0.  , 0.16, 0.  , 0.78],
       [0.  , 0.  , 0.98, 0.  , 0.01, 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]],
      dtype=float32)

In [25]:
yPred = np.argmax(yProbaMc, axis=1)
yPred

array([9, 2, 1])

Alpha Dropout

For selu

In [26]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation='softmax')
])

In [27]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [29]:
train = model.fit(xTrainScaled, yTrain, epochs=2 ,validation_data=(xValidScaled, yValid))

Epoch 1/2
1407/1407 [==============================] - 4s 3ms/step - loss: 0.6690 - accuracy: 0.7609 - val_loss: 0.5866 - val_accuracy: 0.8285
Epoch 2/2
1407/1407 [==============================] - 4s 3ms/step - loss: 0.5088 - accuracy: 0.8112 - val_loss: 0.4961 - val_accuracy: 0.8527


Max Norm Regularization

In [33]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation='selu', kernel_initializer='lecun_normal' ,kernel_constraint=keras.constraints.max_norm(1.)),
    keras.layers.AlphaDropout(rate=0.2,),
    keras.layers.Dense(100, activation='selu', kernel_initializer='lecun_normal', kernel_constraint=keras.constraints.max_norm(1.)),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation='softmax')
])

In [34]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='nadam', metrics=['accuracy'])

In [35]:
train = model.fit(xTrainScaled, yTrain, epochs=2,validation_data=(xValidScaled, yValid))

Epoch 1/2
1407/1407 [==============================] - 5s 3ms/step - loss: 0.6633 - accuracy: 0.7643 - val_loss: 0.5751 - val_accuracy: 0.8270
Epoch 2/2
1407/1407 [==============================] - 4s 3ms/step - loss: 0.5164 - accuracy: 0.8092 - val_loss: 0.5076 - val_accuracy: 0.8463
